<a href="https://colab.research.google.com/github/nyanta012/chatgpt_api_practice/blob/main/section6/section6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratoryのショートカットキー
1. **現在のセルを実行し、次のセルを選択**: `Shift + Enter`
2. **コードセルをテキストセルにする**: `Ctrl + M M`
3. **コードセルを上に追加**: `Ctrl + M A`
4. **コードセルを下に追加**: `Ctrl + M B`
5. **セルの削除**: `Ctrl + M D`

# API KEYの設定

In [ ]:
%%capture
!pip install openai==1.3.5

In [ ]:
import getpass
import json

from openai import OpenAI
from IPython.display import Markdown, display

apikey = getpass.getpass(prompt="OpenAIのAPIキーを入力してください")
client = OpenAI(api_key=apikey)

OpenAIのAPIキーを入力してください··········


# 並行処理で高速に回答を得よう

並行処理とは簡単に言うと、並行して異なる処理を行うこと

**例えば、料理を作るケースを考えると･･･**  
何かの材料をオーブンで温めている待ち時間に、別の材料を切ったりするイメージ

**今回の例だと･･･**  
 APIのリクエストを送ってから返答を得るまで(オーブンで温めている待ち時間)  
 次のAPIのリクエストを送る(別の材料を切ったりする)と高速化できる

In [ ]:
def get_chatgpt_response(
    user_input: str,
    template: str,
    model: str = "gpt-3.5-turbo-1106",
    temperature: float = 0,
    max_tokens: int = 500,
) -> str:
    """
    ChatGPTに対して対話を投げかけ、返答を取得する
    """
    prompt = template.format(user_input=user_input)
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        seed=0
    )
    return response.choices[0].message.content

In [ ]:
PROMPT_TEMPLATE = """
次の文章を英語にしてください

{user_input}

"""

In [ ]:
%%time
get_chatgpt_response("こんにちは", PROMPT_TEMPLATE)

CPU times: user 31 ms, sys: 1.24 ms, total: 32.2 ms
Wall time: 624 ms


'Hello.'

In [ ]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
%%time

# スレッドプールを作成し、最大3つのスレッドを同時に実行できるようにする
list_user_input = ["こんにちは", "こんばんは", "おはよう"]
tpe = ThreadPoolExecutor(max_workers=len(list_user_input))

list_response = []
for user_input in list_user_input:
    response = tpe.submit(get_chatgpt_response, user_input, PROMPT_TEMPLATE)
    list_response.append(response)
tpe.shutdown()  # 全てのタスクが実行されるとスレッドが終了する

# 結果を表示
for r in list_response:
    print(r.result())

Hello.
Good evening.
Good morning.
CPU times: user 29.8 ms, sys: 3.74 ms, total: 33.5 ms
Wall time: 525 ms


# Section5のメール生成を並行処理で実装してみよう

In [ ]:
def get_chatgpt_response(
    user_input: str,
    category_1: str,
    category_2: str,
    template: str,
    model: str = "gpt-3.5-turbo-1106",
    temperature: float = 0,
    max_tokens: int = 500,
) -> str:
    """
    ChatGPTに対して対話を投げかけ、返答を取得する
    """
    prompt = template.format(
        user_input=user_input, category_1=category_1, category_2=category_2
    )
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        seed=0
    )
    return response.choices[0].message.content

In [ ]:
PROMPT_TEMPLATE = """
    あなたは優秀なカスタマーサポートセンターの担当者です。
    あなたの仕事は下記の顧客の問い合わせとカテゴリーに対して実行手順を処理することで、
    適切な部門のメールアドレスを顧客に伝えるためのメールを作成することです。
    下記の実行手順に従って段階的に考えて、最高の品質でメールの文章を作成してください。

    # 顧客の問い合わせ
     {user_input}

    # カテゴリー
      第一カテゴリー:{category_1}
      第二カテゴリー:{category_2}

    # 実行手順
    1. 第一カテゴリーと第二カテゴリーに応じてメールアドレスを下記から抽出してください。
       下記は第一カテゴリー：第二カテゴリーを表す。
         製品：スペック は product_spec@example.com
         製品：値段 は product_price@example.com
         製品：購入方法 は product_purchase_method@example.com
         注文：注文状況 は order_status@example.com
         注文：注文変更 は order_change@example.com
         注文：支払い方法 は order_payment_method@example.com
         配送：配送ステータス は delivery_status@example.com
         配送：配送オプション は delivery_option@example.com
         配送：配送問題 は delivery_issue@example.com

    2. メール文章を作成
      顧客からの問い合わせに対して、カスタマーセンターの従業員として責任をもって丁寧な文章で
      1で抽出したメールアドレスに問い合わせをお願いする文章を300文字程度で作成する

    # メール文章形式
      考えたメールタイトル
      考えたメール本文
    """

In [ ]:
import pandas as pd

inquiries = [
    "このカメラの最大シャッタースピードは何ですか？",
    "私のパッケージの追跡番号を教えてください。",
    "エクスプレス配送は可能ですか？",
    "特定の日時に配送することはできますか？",
    "配送された商品が損傷していました。どうすればいいですか？",
    "注文の支払いにApple PayやGoogle Payは使用できますか？",
    "私のパッケージはいつ到着予定ですか？",
    "注文の支払いにクレジットカードは使用できますか？",
    "私の注文はまだ出荷されていませんか？",
    "この製品はオンラインで注文することができますか？",
]

list_json = [
    {"key": "製品", "value": "スペック"},
    {"key": "配送", "value": "配送状況"},
    {"key": "配送", "value": "配送オプション"},
    {"key": "配送", "value": "配送問題"},
    {"key": "配送", "value": "配送問題"},
    {"key": "注文", "value": "支払い方法"},
    {"key": "配送", "value": "配送状況"},
    {"key": "注文", "value": "支払い方法"},
    {"key": "配送", "value": "注文状況"},
    {"key": "注文", "value": "購入方法"},
]

df = pd.DataFrame(list_json)
df["inquiry"] = inquiries
df.columns = ["category_1", "category_2", "inquiry"]

In [ ]:
%%time
# スレッドプールを作成
tpe = ThreadPoolExecutor(max_workers=len(df))

list_response = []
for _, row in df.iterrows():
    category_1, category_2, inquiry = row['category_1'], row['category_2'], row['inquiry']
    response = tpe.submit(
        get_chatgpt_response, inquiry, category_1, category_2, PROMPT_TEMPLATE
    )
    list_response.append(response)
tpe.shutdown()  # 全てのタスクが実行されるとスレッドが終了する


# 結果を表示
for i, r in enumerate(list_response):
    inquiry = df.loc[i, "inquiry"]
    display(
        Markdown(
            f"""\n\n### メール {i+1} \n **問い合わせ内容:** {inquiry}\n\n **生成文:**\n\n {r.result()}"""
        )
    )



### メール 1 
 **問い合わせ内容:** このカメラの最大シャッタースピードは何ですか？

 **生成文:**

 件名：カメラの最大シャッタースピードについての問い合わせ

いつもお世話になっております。カスタマーサポートセンターの担当者です。

お客様からの問い合わせに対して、製品のスペックに関するお問い合わせを受け付けました。お手数ですが、以下のメールアドレスに直接お問い合わせいただけますでしょうか。

製品：スペック は product_spec@example.com

カメラの最大シャッタースピードについての詳細な情報をお客様にお伝えできる部署になりますので、お手数ですがそちらにお問い合わせいただけますと幸いです。

何かご不明点がございましたら、お気軽にお知らせください。何卒よろしくお願い申し上げます。

以上、よろしくお願いいたします。

カスタマーサポートセンターの担当者より



### メール 2 
 **問い合わせ内容:** 私のパッケージの追跡番号を教えてください。

 **生成文:**

 件名：パッケージの追跡番号についてのお問い合わせ

いつも弊社のサービスをご利用いただき、誠にありがとうございます。お客様からのパッケージの追跡番号についてのお問い合わせにつきまして、お手数をおかけいたしますが、以下のメールアドレスに直接お問い合わせいただけますでしょうか。

配送：配送ステータス は delivery_status@example.com

お手数をおかけいたしますが、上記のメールアドレスにお問い合わせいただければ、迅速に対応させていただきます。何かご不明点がございましたら、お気軽にお知らせください。

何卒よろしくお願い申し上げます。

株式会社〇〇 カスタマーサポートセンター



### メール 3 
 **問い合わせ内容:** エクスプレス配送は可能ですか？

 **生成文:**

 件名：エクスプレス配送についてのお問い合わせ

いつも弊社のカスタマーサポートセンターにお問い合わせいただき、誠にありがとうございます。

お客様からのお問い合わせについて、配送オプションに関するご質問を承りました。お客様のご要望にお応えするために、配送オプションに関する詳細な情報をお知らせいただけますでしょうか。

この件については、配送カテゴリーに関するお問い合わせとなりますので、配送オプション担当のメールアドレスにお問い合わせいただくことが適切です。配送オプション担当のメールアドレスは delivery_option@example.com です。

お手数をおかけいたしますが、上記のメールアドレス宛に直接お問い合わせいただけますようお願い申し上げます。

何かご不明点がございましたら、お気軽にお知らせください。お客様のご要望にお応えできるよう、全力でサポートさせていただきます。

何卒よろしくお願い申し上げます。

カスタマーサポートセンター
[会社名]



### メール 4 
 **問い合わせ内容:** 特定の日時に配送することはできますか？

 **生成文:**

 件名：特定の日時に配送することについてのお問い合わせ

いつも弊社の配送サービスをご利用いただき、誠にありがとうございます。お客様からいただいたお問い合わせについて、お手伝いさせていただきます。

お問い合わせ内容に関しては、配送カテゴリーに該当いたします。そのため、配送に関するお問い合わせについては、下記のメールアドレスにご連絡いただけますようお願い申し上げます。

配送問題に関するお問い合わせについては、delivery_issue@example.com までご連絡ください。

お客様のご要望にできる限りお応えできるよう、スタッフ一同努めてまいります。何かご不明点やご質問がございましたら、お気軽にお知らせください。

何かお手伝いできることがございましたら、お知らせください。引き続き、弊社のサービスをご利用いただけますようお願い申し上げます。

どうぞよろしくお願いいたします。

カスタマーサポートセンターより



### メール 5 
 **問い合わせ内容:** 配送された商品が損傷していました。どうすればいいですか？

 **生成文:**

 件名：配送された商品の損傷についての問い合わせ

いつも弊社の商品をご利用いただき、誠にありがとうございます。お客様からの配送された商品が損傷していたというお問い合わせについて、大変申し訳ございません。

このような問題については、配送部門の担当者が最も適切に対応させていただきます。そのため、お手数ですが配送問題に関するご連絡は、delivery_issue@example.comまでお送りいただけますでしょうか。

お手続きに関しては迅速に対応させていただきますので、何卒よろしくお願い申し上げます。

何かご不明点がございましたら、お気軽にお知らせください。

敬具

カスタマーサポートセンター

...



### メール 7 
 **問い合わせ内容:** 私のパッケージはいつ到着予定ですか？

 **生成文:**

 件名：パッケージ到着予定についての問い合わせ

いつも弊社のサービスをご利用いただき、誠にありがとうございます。お客様からのお問い合わせについて、配送部門にお問い合わせいただく必要がございます。

お手数をおかけいたしますが、以下のメールアドレスに直接お問い合わせいただけますでしょうか。
配送：配送ステータス は delivery_status@example.com

お客様のご理解とご協力に感謝申し上げます。何かご不明点がございましたら、お気軽にお知らせください。

よろしくお願いいたします。

カスタマーサポートセンター



### メール 8 
 **問い合わせ内容:** 注文の支払いにクレジットカードは使用できますか？

 **生成文:**

 件名：注文支払いについての問い合わせ

いつも弊社製品をご利用いただき、誠にありがとうございます。お客様からの注文支払いに関するお問い合わせについて、お手伝いさせていただきます。

お問い合わせ内容については、注文カテゴリーの支払い方法に関するものとなります。そのため、以下のメールアドレスにお問い合わせいただけますようお願い申し上げます。

注文：支払い方法
メールアドレス：order_payment_method@example.com

お手数をおかけいたしますが、上記のメールアドレス宛にお問い合わせいただけますと、迅速にご対応させていただきます。

何かご不明点がございましたら、お気軽にお知らせください。引き続き、弊社製品をご愛顧いただけますようお願い申し上げます。

よろしくお願いいたします。

カスタマーサポートセンター
[会社名]



### メール 9 
 **問い合わせ内容:** 私の注文はまだ出荷されていませんか？

 **生成文:**

 件名：注文の配送状況についてのお問い合わせ

いつもご利用いただき、誠にありがとうございます。お客様の注文の配送状況についてのお問い合わせを承りました。

お客様の注文に関する配送状況については、配送ステータスに関するお問い合わせは、delivery_status@example.com までご連絡いただけます。

お手数をおかけいたしますが、上記のメールアドレスに直接お問い合わせいただければ、迅速に対応させていただきます。

何かご不明点がございましたら、お気軽にお知らせください。引き続き、ご利用いただきありがとうございます。

よろしくお願いいたします。

カスタマーサポートセンター



### メール 10 
 **問い合わせ内容:** この製品はオンラインで注文することができますか？

 **生成文:**

 件名：オンラインでの製品注文についての問い合わせ

いつも弊社製品にご興味をお持ちいただき、誠にありがとうございます。お問い合わせいただきました「オンラインでの製品注文」について、お手数ですが以下のメールアドレスにご連絡いただけますでしょうか。

製品：購入方法
メールアドレス：product_purchase_method@example.com

ご不明点やご質問がございましたら、遠慮なくお知らせください。お客様のご要望にお応えできるよう、心を込めてサポートさせていただきます。何卒よろしくお願い申し上げます。

株式会社〇〇
カスタマーサポートセンター

CPU times: user 176 ms, sys: 16 ms, total: 192 ms
Wall time: 5.77 s
